In [12]:
!pip3 install seaborn bs4 textblob lxml tflearn


    100% |################################| 102kB 3.5MB/s a 0:00:011
  Running setup.py bdist_wheel for tflearn ... done
  Stored in directory: /root/.cache/pip/wheels/d0/f6/69/0ef3ee395aac2e5d15d89efd29a9a216f3c27767b43b72c006
Successfully built tflearn


In [13]:
#https://tm3.ghost.io/2017/04/19/amazon-food-reviews-part-i/
#https://tm3.ghost.io/2017/04/21/amazon-food-reviews-part-vi/

from __future__ import division
from __future__ import print_function
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pylab
from bs4 import BeautifulSoup
import textblob
from textblob import TextBlob
import pickle
import datetime
from scipy.stats import kendalltau
data = pd.read_csv("./Review100.csv")
print(data.dtypes)


Id                         int64
ProductId                 object
UserId                    object
ProfileName               object
HelpfulnessNumerator       int64
HelpfulnessDenominator     int64
Score                      int64
Time                       int64
Summary                   object
Text                      object
dtype: object


In [14]:
#clean text using bs4 - approx. 10 minutes
data['text_cln']= data['Text'].map(lambda x: BeautifulSoup(x, "html.parser").get_text())

#apply the polarity score to each text feature using 
#TextBlob ()

data['tb_polarity']= data['text_cln'].map(lambda x: 
TextBlob(x).sentiment.polarity)

#pickle
data.to_pickle('amz_data.pkl')




In [15]:
data = pd.read_pickle('amz_data.pkl')
print(data)

    Id   ProductId          UserId                      ProfileName  \
0    1  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian   
1    2  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2    3  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   
3    4  B000UA0QIQ  A395BORC6FGVXV                             Karl   
4    5  B006K2ZZ7K  A1UQRSCLF8GW1T    Michael D. Bigham "M. Wassir"   
5    6  B006K2ZZ7K   ADT0SRK1MGOEU                   Twoapennything   
6    7  B006K2ZZ7K  A1SP2KVKFXXRU1                David C. Sullivan   
7    8  B006K2ZZ7K  A3JRGQVEQN31IQ               Pamela G. Williams   
8    9  B000E7L2R4  A1MZYO9TZK0BBI                         R. James   
9   10  B00171APVA  A21BT40VZCCYT4                    Carol A. Reed   
10  11  B0001PB9FE  A3HDKO7OW0QNK4                     Canadian Fan   
11  12  B0009XLVG0  A2725IB4YY9JEB           A Poeng "SparkyGoHome"   
12  13  B0009XLVG0   A327PCT23YH90                               LT   
13  14

In [16]:
import time
from time import time
import keras
from keras.datasets import reuters
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
import tensorflow as tf
from keras.models import load_model

#Read in data
#read pickle from previous blog post Part I

data = pd.read_pickle('amz_data.pkl')

#normalize date time
data2 = data.copy()
data2['datetime'] = data2['Time'].map(lambda x: (datetime.datetime.fromtimestamp(int(x)).strftime('%Y-%m-%d %H:%M:%S')))
data2['datetime'] = pd.to_datetime(data2['datetime'])


In [40]:
#train/test split 80/20
from sklearn.model_selection import train_test_split
X_data, y_target = data2['text_cln'], data2['Score']
y_target = y_target.values
X_datatrain, X_datatest, y_train, y_test = train_test_split(X_data, y_target, test_size=0.2, random_state=0)


In [41]:
# Process vocabulary  
import time
import tflearn

MAX_DOCUMENT_LENGTH = 800
vocab_processor = tflearn.data_utils.VocabularyProcessor(MAX_DOCUMENT_LENGTH)
start_time = time.time() #timing it 

x_train = np.array(list(vocab_processor.fit_transform(X_datatrain))) 
print("--- %s seconds ---" % (time.time() - start_time))  

x_test = np.array(list(vocab_processor.transform(X_datatest)))
print("--- %s seconds ---" % (time.time() - start_time))  

n_words = len(vocab_processor.vocabulary_)
print("--- %s seconds ---" % (time.time() - start_time))  

print('Total words: %d' % n_words)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

#confirm that the shape is consistent with max_document_lenght = 800
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

--- 0.022951602935791016 seconds ---
--- 0.02631521224975586 seconds ---
--- 0.026639699935913086 seconds ---
Total words: 1560
79 train sequences
20 test sequences
x_train shape: (79, 800)
x_test shape: (20, 800)


In [56]:
max_words = MAX_DOCUMENT_LENGTH
batch_size = 32
#An epoch is a full pass over your training data
#this one will pass over the training set 5 times
#In 32 batches
epochs = 500

In [57]:
#number of target classes

num_classes = int(np.max(y_train))+1
print(num_classes, 'classes')
'''
print('Convert class vector to binary class matrix '
  '(for use with categorical_crossentropy)')
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

'''
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)
print(y_test)

6 classes
y_train shape: (79,)
y_test shape: (20,)
[1 5 4 5 1 5 2 1 4 5 3 5 3 4 5 5 5 5 4 5]


In [58]:
print('Building model...')
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(512, input_shape=(max_words,)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(num_classes))
model.add(tf.keras.layers.Activation('softmax'))

Building model...


In [59]:
model.compile(loss='sparse_categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print("--- %s seconds ---" % (time.time() - start_time))  

#timed it at around 8 minutes
history = model.fit(x_train, y_train,
                batch_size=batch_size,
                epochs=epochs,
                verbose=1,
                validation_split=0.1)
print("--- %s seconds ---" % (time.time() - start_time)) 

--- 377.217627286911 seconds ---
Train on 71 samples, validate on 8 samples
Epoch 1/500
71/71 [==============================] - 0s 3ms/sample - loss: 12.6663 - acc: 0.1690 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 2/500
71/71 [==============================] - 0s 576us/sample - loss: 7.2810 - acc: 0.5352 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 3/500
71/71 [==============================] - 0s 528us/sample - loss: 5.0172 - acc: 0.6761 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 4/500
71/71 [==============================] - 0s 592us/sample - loss: 4.5039 - acc: 0.7042 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 5/500
71/71 [==============================] - 0s 703us/sample - loss: 4.3133 - acc: 0.7324 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 6/500
71/71 [==============================] - 0s 568us/sample - loss: 4.2370 - acc: 0.7324 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 7/500
71/71 [==============================] - 0s 557us/sample - loss: 4.3133 - acc: 0.7324 - val_loss: 6.044

Epoch 61/500
71/71 [==============================] - 0s 455us/sample - loss: 4.3133 - acc: 0.7324 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 62/500
71/71 [==============================] - 0s 529us/sample - loss: 4.3133 - acc: 0.7324 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 63/500
71/71 [==============================] - 0s 546us/sample - loss: 4.3133 - acc: 0.7324 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 64/500
71/71 [==============================] - 0s 512us/sample - loss: 4.1682 - acc: 0.7324 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 65/500
71/71 [==============================] - 0s 718us/sample - loss: 4.2566 - acc: 0.7324 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 66/500
71/71 [==============================] - 0s 672us/sample - loss: 4.1894 - acc: 0.7324 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 67/500
71/71 [==============================] - 0s 795us/sample - loss: 4.0863 - acc: 0.7465 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 68/500
71/71 [============================

71/71 [==============================] - 0s 618us/sample - loss: 4.0863 - acc: 0.7465 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 122/500
71/71 [==============================] - 0s 615us/sample - loss: 4.0863 - acc: 0.7465 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 123/500
71/71 [==============================] - 0s 693us/sample - loss: 4.0863 - acc: 0.7465 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 124/500
71/71 [==============================] - 0s 542us/sample - loss: 4.0863 - acc: 0.7465 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 125/500
71/71 [==============================] - 0s 613us/sample - loss: 4.0863 - acc: 0.7465 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 126/500
71/71 [==============================] - 0s 559us/sample - loss: 4.0863 - acc: 0.7465 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 127/500
71/71 [==============================] - 0s 537us/sample - loss: 4.3133 - acc: 0.7324 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 128/500
71/71 [==============================] - 

71/71 [==============================] - 0s 814us/sample - loss: 4.0863 - acc: 0.7465 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 181/500
71/71 [==============================] - 0s 728us/sample - loss: 4.0863 - acc: 0.7465 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 182/500
71/71 [==============================] - 0s 752us/sample - loss: 4.0863 - acc: 0.7465 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 183/500
71/71 [==============================] - 0s 659us/sample - loss: 4.0863 - acc: 0.7465 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 184/500
71/71 [==============================] - 0s 719us/sample - loss: 4.0863 - acc: 0.7465 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 185/500
71/71 [==============================] - 0s 630us/sample - loss: 4.0863 - acc: 0.7465 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 186/500
71/71 [==============================] - 0s 602us/sample - loss: 4.0863 - acc: 0.7465 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 187/500
71/71 [==============================] - 

Epoch 240/500
71/71 [==============================] - 0s 527us/sample - loss: 4.0863 - acc: 0.7465 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 241/500
71/71 [==============================] - 0s 571us/sample - loss: 4.0863 - acc: 0.7465 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 242/500
71/71 [==============================] - 0s 545us/sample - loss: 4.0863 - acc: 0.7465 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 243/500
71/71 [==============================] - 0s 545us/sample - loss: 4.0863 - acc: 0.7465 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 244/500
71/71 [==============================] - 0s 624us/sample - loss: 4.0863 - acc: 0.7465 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 245/500
71/71 [==============================] - 0s 468us/sample - loss: 4.0863 - acc: 0.7465 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 246/500
71/71 [==============================] - 0s 646us/sample - loss: 4.0863 - acc: 0.7465 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 247/500
71/71 [====================

Epoch 300/500
71/71 [==============================] - 0s 525us/sample - loss: 4.0863 - acc: 0.7465 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 301/500
71/71 [==============================] - 0s 584us/sample - loss: 4.0863 - acc: 0.7465 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 302/500
71/71 [==============================] - 0s 622us/sample - loss: 4.0863 - acc: 0.7465 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 303/500
71/71 [==============================] - 0s 691us/sample - loss: 4.0863 - acc: 0.7465 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 304/500
71/71 [==============================] - 0s 551us/sample - loss: 4.0863 - acc: 0.7465 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 305/500
71/71 [==============================] - 0s 681us/sample - loss: 4.0863 - acc: 0.7465 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 306/500
71/71 [==============================] - 0s 695us/sample - loss: 4.0863 - acc: 0.7465 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 307/500
71/71 [====================

Epoch 360/500
71/71 [==============================] - 0s 538us/sample - loss: 4.0863 - acc: 0.7465 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 361/500
71/71 [==============================] - 0s 608us/sample - loss: 4.0863 - acc: 0.7465 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 362/500
71/71 [==============================] - 0s 473us/sample - loss: 4.0863 - acc: 0.7465 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 363/500
71/71 [==============================] - 0s 519us/sample - loss: 4.0863 - acc: 0.7465 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 364/500
71/71 [==============================] - 0s 588us/sample - loss: 4.0863 - acc: 0.7465 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 365/500
71/71 [==============================] - 0s 582us/sample - loss: 4.0863 - acc: 0.7465 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 366/500
71/71 [==============================] - 0s 607us/sample - loss: 4.0863 - acc: 0.7465 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 367/500
71/71 [====================

Epoch 420/500
71/71 [==============================] - 0s 563us/sample - loss: 4.3133 - acc: 0.7324 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 421/500
71/71 [==============================] - 0s 506us/sample - loss: 4.3133 - acc: 0.7324 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 422/500
71/71 [==============================] - 0s 603us/sample - loss: 4.3133 - acc: 0.7324 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 423/500
71/71 [==============================] - 0s 559us/sample - loss: 4.3133 - acc: 0.7324 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 424/500
71/71 [==============================] - 0s 499us/sample - loss: 4.3133 - acc: 0.7324 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 425/500
71/71 [==============================] - 0s 533us/sample - loss: 4.3133 - acc: 0.7324 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 426/500
71/71 [==============================] - 0s 521us/sample - loss: 4.3133 - acc: 0.7324 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 427/500
71/71 [====================

71/71 [==============================] - 0s 687us/sample - loss: 4.3133 - acc: 0.7324 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 480/500
71/71 [==============================] - 0s 825us/sample - loss: 4.3133 - acc: 0.7324 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 481/500
71/71 [==============================] - 0s 846us/sample - loss: 4.3133 - acc: 0.7324 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 482/500
71/71 [==============================] - 0s 769us/sample - loss: 4.3133 - acc: 0.7324 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 483/500
71/71 [==============================] - 0s 665us/sample - loss: 4.3133 - acc: 0.7324 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 484/500
71/71 [==============================] - 0s 782us/sample - loss: 4.3133 - acc: 0.7324 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 485/500
71/71 [==============================] - 0s 917us/sample - loss: 4.3133 - acc: 0.7324 - val_loss: 6.0443 - val_acc: 0.6250
Epoch 486/500
71/71 [==============================] - 

In [60]:
score = model.evaluate(x_test, y_test,
                   batch_size=batch_size, verbose=1)
print("--- %s seconds ---" % (time.time() - start_time))  

print('Test score:', score[0])
print('Test accuracy:', score[1])

20/20 [==============================] - 0s 106us/sample - loss: 8.0590 - acc: 0.5000
--- 422.47250962257385 seconds ---
Test score: 8.059048652648926
Test accuracy: 0.5


In [55]:
model.predict_classes(x_test, batch_size=batch_size, verbose=1)
model.predict_proba(x_test, batch_size=batch_size, verbose=1)

20/20 [==============================] - 0s 84us/sample


array([[0.0000000e+00, 2.6213012e-03, 0.0000000e+00, 9.9737871e-01,
        0.0000000e+00, 7.3947072e-19],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 1.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 1.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 1.0000000e+00],
       [2.0954903e-16, 1.8145166e-12, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 1.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 3.2335971e-38, 5.1122812e-14,
        0.0000000e+00, 1.0000000e+00],
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 1.3342836e-12],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 1.0000000e+00],
       [0.0000000e+00, 0.0000000e+00,